In [1]:
!pip install pycryptodome psutil memory-profiler

Defaulting to user installation because normal site-packages is not writeable


In [4]:
import os
import time
import psutil
from ccakem import kem_keygen512, kem_encaps512, kem_decaps512
from ccakem import kem_keygen768, kem_encaps768, kem_decaps768
from ccakem import kem_keygen1024, kem_encaps1024, kem_decaps1024
from binascii import unhexlify, hexlify
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from util import cast_to_byte

# Função para converter os dados para hexadecimal com máscara para garantir que os valores estejam entre 0 e 255
def safe_hexlify(data):
    if isinstance(data, list):
        data = [(x & 0xFF) for x in data]
    return hexlify(bytearray(data)).decode()

# Função para converter uma lista de inteiros em bytes
def list_to_bytes(data):
    return bytes([x & 0xFF for x in data])

# Função para salvar resultados do benchmark em um arquivo .txt
def save_benchmark_results_to_file(filename, level, repetitions, results):
    with open(filename, 'a') as f:
        f.write("\n===== Resultados do Benchmark =====\n")
        f.write(f"Tipo de Criptografia: Kyber-{level}\n")
        f.write(f"Quantidade de execuções: {repetitions}\n")
        for key, value in results.items():
            f.write(f'{key}: {value}\n')
        f.write("=====================================\n")

# Função para salvar resultados da criptografia em um arquivo .txt
def save_crypto_results_to_file(filename, level, results):
    with open(filename, 'a') as f:
        f.write("\n===== Resultados da Criptografia =====\n")
        f.write(f"Tipo de Criptografia: Kyber-{level}\n")
        for key, value in results.items():
            f.write(f'{key}: "{value}"\n')
        f.write("=====================================\n")

# Função para criptografar o texto usando AES
def aes_encrypt(shared_secret, plaintext):
    key = list_to_bytes(shared_secret[:16])  # Truncando o Shared Secret para 16 bytes
    print(f"Chave AES usada para criptografar (hex): {hexlify(key).decode()}")
    
    nonce = get_random_bytes(12)  # Gerando nonce de 12 bytes
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    
    ciphertext, tag = cipher.encrypt_and_digest(plaintext)
    return nonce, ciphertext, tag

# Função para descriptografar o texto usando AES
def aes_decrypt(shared_secret, nonce, ciphertext, tag):
    key = list_to_bytes(shared_secret[:16])  # Truncando o Shared Secret para 16 bytes
    print(f"Chave AES usada para descriptografar (hex): {hexlify(key).decode()}")
    
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    plaintext = cipher.decrypt_and_verify(ciphertext, tag)
    return plaintext

# Função para criptografar um texto com Kyber e AES
def kyber_encrypt(level, plaintext):
    if level == 512:
        priv_key, pub_key = kem_keygen512()
        shared_secret, ciphertext_kem = kem_encaps512(pub_key)
    elif level == 768:
        priv_key, pub_key = kem_keygen768()
        shared_secret, ciphertext_kem = kem_encaps768(pub_key)
    elif level == 1024:
        priv_key, pub_key = kem_keygen1024()
        shared_secret, ciphertext_kem = kem_encaps1024(pub_key)
    else:
        print("Nível de criptografia inválido. Selecione 512, 768 ou 1024.")
        return None, None, None, None, None

    nonce, ciphertext, tag = aes_encrypt(shared_secret, plaintext)  # Criptografa o texto com AES

    # Salvar resultados da criptografia
    results = {
        "Private Key (hex)": safe_hexlify(priv_key),
        "Public Key (hex)": safe_hexlify(pub_key),
        "Shared Secret (hex)": safe_hexlify(shared_secret),
        "Ciphertext KEM (hex)": safe_hexlify(ciphertext_kem),
        "Nonce (hex)": safe_hexlify(nonce),
        "Ciphertext AES (hex)": safe_hexlify(ciphertext),
        "Tag (hex)": safe_hexlify(tag)
    }
    save_crypto_results_to_file('kyber_cryptography_results.txt', level, results)

    print("\n===== Criptografia Completa =====")
    for key, value in results.items():
        print(f'{key}: "{value}"')

    return priv_key, ciphertext_kem, nonce, ciphertext, tag

# Função para descriptografar o texto com Kyber e AES
def kyber_decrypt(level, priv_key, ciphertext_kem, nonce, ciphertext, tag):
    if level == 512:
        shared_secret_dec = kem_decaps512(priv_key, ciphertext_kem)
    elif level == 768:
        shared_secret_dec = kem_decaps768(priv_key, ciphertext_kem)
    elif level == 1024:
        shared_secret_dec = kem_decaps1024(priv_key, ciphertext_kem)
    else:
        print("Nível de criptografia inválido. Selecione 512, 768 ou 1024.")
        return
    
    print(f"Shared Secret Decapsulado (hex): {safe_hexlify(shared_secret_dec)}")

    # Tenta descriptografar o texto usando AES com a chave compartilhada decapsulada
    try:
        plaintext = aes_decrypt(shared_secret_dec, nonce, ciphertext, tag)
        print("\n===== Descriptografia Completa =====")
        print(f"Texto descriptografado: {plaintext.decode()}")
    except ValueError as e:
        print(f"Erro na verificação do MAC: {e}")
        print("Verifique se todas as informações inseridas (chaves, nonce, ciphertext e tag) estão corretas.")

# Função para medir tempo, processamento e memória com psutil
def benchmark(function, *args):
    start_time = time.time()  # Mede o tempo
    process = psutil.Process(os.getpid())  # Processo atual
    cpu_before = process.cpu_percent(interval=None) / psutil.cpu_count(logical=False)  # CPU antes por núcleo
    mem_before = process.memory_info().rss / 1024 / 1024  # Memória antes

    result = function(*args)  # Executa a função

    end_time = time.time()  # Tempo final
    cpu_after = process.cpu_percent(interval=None) / psutil.cpu_count(logical=False)  # CPU depois por núcleo
    mem_after = process.memory_info().rss / 1024 / 1024  # Memória depois

    # Calcula métricas de desempenho
    time_elapsed = end_time - start_time
    cpu_used = cpu_after - cpu_before
    mem_used = mem_after - mem_before

    # Retorna métricas para análise
    return time_elapsed, cpu_used, mem_used, result

# Função para criptografar e descriptografar repetidamente
def test_encrypt_decrypt_stress():
    print("\n=== Teste de Criptografia e Descriptografia Repetida ===")

    # Solicita o nível de criptografia, o texto e o número de repetições
    level = int(input("Selecione o nível de criptografia (512, 768, 1024): "))
    plaintext = input("Digite o texto que deseja criptografar: ").encode('utf-8')
    repetitions = int(input("Quantas vezes deseja repetir o teste?: "))

    # Variáveis para armazenar métricas
    total_time = 0
    total_cpu = 0
    total_mem = 0
    max_cpu = 0
    max_mem = 0

    for i in range(repetitions):
        print(f"\n--- Iteração {i+1} de {repetitions} ---")
        
        # Criptografia
        time_elapsed, cpu_used, mem_used, (priv_key, ciphertext_kem, nonce, ciphertext, tag) = benchmark(kyber_encrypt, level, plaintext)
        
        if priv_key is None:
            return

        # Descriptografia
        time_elapsed_dec, cpu_used_dec, mem_used_dec, _ = benchmark(kyber_decrypt, level, priv_key, ciphertext_kem, nonce, ciphertext, tag)

        # Atualiza as métricas
        total_time += (time_elapsed + time_elapsed_dec)
        total_cpu += (cpu_used + cpu_used_dec)
        total_mem += (mem_used + mem_used_dec)

        # Verifica máximo de CPU e Memória
        max_cpu = max(max_cpu, cpu_used, cpu_used_dec)
        max_mem = max(max_mem, mem_used, mem_used_dec)

    # Cálculo de métricas médias
    avg_cpu = total_cpu / (2 * repetitions)
    avg_mem = total_mem / (2 * repetitions)

    # Salvar os resultados do benchmark
    benchmark_results = {
        "Tempo total": f"{total_time:.4f} segundos",
        "Consumo máximo de CPU": f"{max_cpu:.4f}%",
        "Consumo máximo de Memória": f"{max_mem:.4f} MiB",
        "Média de consumo de CPU": f"{avg_cpu:.4f}%",
        "Média de consumo de Memória": f"{avg_mem:.4f} MiB"
    }
    save_benchmark_results_to_file('kyber_benchmark_results.txt', level, repetitions, benchmark_results)

    # Exibe análise final
    print("\n===== Análise Final =====")
    print(f"Tempo total: {total_time:.4f} segundos")
    print(f"Consumo máximo de CPU: {max_cpu:.4f}%")
    print(f"Consumo máximo de Memória: {max_mem:.4f} MiB")
    print(f"Média de consumo de CPU: {avg_cpu:.4f}%")
    print(f"Média de consumo de Memória: {avg_mem:.4f} MiB")

# Função principal que permite a seleção do usuário
def main():
    print("Selecione uma opção:")
    print("1: Criptografar um texto usando Kyber e AES")
    print("2: Descriptografar um texto usando Kyber e AES")
    print("3: Testar Criptografia e Descriptografia em uma única etapa com benchmark")
    #print("4: Testar Criptografia e Descriptografia repetidamente para estresse de processamento")
    
    choice = input("Digite o número da opção desejada: ")
    
    if choice == '1':
        print("\n=== Criptografar ===")
        level = int(input("Selecione o nível de criptografia (512, 768, 1024): "))
        plaintext = input("Digite o texto que deseja criptografar: ").encode('utf-8')
        kyber_encrypt(level, plaintext)
    elif choice == '2':
        print("\n=== Descriptografar ===")
        level = int(input("Selecione o nível de criptografia (512, 768, 1024): "))
        priv_key_hex = input("Digite a Private Key em hex: ")
        ciphertext_kem_hex = input("Digite o Ciphertext KEM em hex: ")
        nonce_hex = input("Digite o Nonce em hex: ")
        ciphertext_hex = input("Digite o Ciphertext AES em hex: ")
        tag_hex = input("Digite o Tag em hex: ")

        priv_key = unhexlify(priv_key_hex)
        ciphertext_kem = unhexlify(ciphertext_kem_hex)
        nonce = unhexlify(nonce_hex)
        ciphertext = unhexlify(ciphertext_hex)
        tag = unhexlify(tag_hex)

        kyber_decrypt(level, priv_key, ciphertext_kem, nonce, ciphertext, tag)
    elif choice == '3':
        print("\n=== Testar Criptografia e Descriptografia com Benchmark ===")
        test_encrypt_decrypt_stress()
    elif choice == '4':
        print("\n=== Testar Criptografia e Descriptografia repetidamente ===")
        test_encrypt_decrypt_stress()
    else:
        print("Escolha inválida.")

if __name__ == "__main__":
    main()


Selecione uma opção:
1: Criptografar um texto usando Kyber e AES
2: Descriptografar um texto usando Kyber e AES
3: Testar Criptografia e Descriptografia em uma única etapa com benchmark


Digite o número da opção desejada:  3



=== Testar Criptografia e Descriptografia com Benchmark ===

=== Teste de Criptografia e Descriptografia Repetida ===


Selecione o nível de criptografia (512, 768, 1024):  512
Digite o texto que deseja criptografar:  Teste de implementação Kyber IME
Quantas vezes deseja repetir o teste?:  100



--- Iteração 1 de 100 ---
Chave AES usada para criptografar (hex): 73d76014ccd1daad8499216ebbd59408

===== Criptografia Completa =====
Private Key (hex): "47b4a56fdba77ef7664a1b588ea267f3e3229ee55fa98123386185007c0beb6648cc777948bc102fdc968d397ca3460855e4b45e88c6a708bdc44cc22069a34a9793446579e22aa979a4b35af8bdc565639a85759e04ac48495d45c06b0c446e69fa2ae20184a139a061689e5886c4d620b7653c1c5c5aa16214cd08687eff6133fb045c1ad5c49aa63e69f221f186ad4bb9a5af52c601f7b3739a31510b7cd68453bd23088747405ff841699a3934f1ba46c859257a9ab8e26a11ecc73b9c5c05e1bc84cb708be6433b5bc719e03fdf79acd697a50d489961cbafc0ab0a6f6026ebd2b08d46505fb640d7c706eb6cc2f8a7012626c10ec513d8175425a47017c34bd6a867e4ea8d5ea69b54704ca083a071638992103b8cc21d5b118f0cc754286a60ffec6722a43b33dcac1497b0ce455bd949ce77884ddfd228e60cb4df22a103e659a6517084c486e0154394b78eed9025ce62c7c5b61a4ec0658588891244979a49499230a3795b0ac4453aeb4a2978b432ed1a68b408841f3c43e450948c64ce307b55efa51a633a1ed85748fe018cff8bab0ae2448fe2a0f3c3a24975c4de4001e6d9